<a target="_blank" href="https://colab.research.google.com/github/shaankhosla/optimizingllms/blob/main/notebooks/PEFT_LoRA.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Source: https://huggingface.co/blog/peft

In [1]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 14.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.2 MB/s eta 0:00:00


In [8]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from transformers import AutoTokenizer, AutoModelForCausalLM

# Here let's load the opt-6.7b model, its weights in half-precision (float16) are about 13GB on the Hub! If we load them in 8-bit we would require around 7GB of memory instead.
model = AutoModelForCausalLM.from_pretrained(
    "distilgpt2",
    device_map="cpu",
)

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
# tokenizer.add_special_tokens({'bos_token': '<bos>',
#                               'eos_token': '<eos>',
#                               'pad_token': '<pad>'})

In [9]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [10]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, lora_alpha=32, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora.py:285: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 294912 || all params: 82207488 || trainable%: 0.35874104315168953


In [11]:
tokenizer.add_special_tokens(
    {"bos_token": "<bos>", "eos_token": "<eos>", "pad_token": "<pad>"}
)

3

In [13]:
import transformers
from datasets import load_dataset

data = load_dataset("Abirate/english_quotes")
data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)


trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=5,
        learning_rate=2e-4,
        # fp16=True,
        logging_steps=1,
        output_dir="outputs",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

Step,Training Loss
1,3.511000
2,3.370100
3,4.049700
4,4.320000
5,3.749900


TrainOutput(global_step=5, training_loss=3.800134801864624, metrics={'train_runtime': 13.1282, 'train_samples_per_second': 1.523, 'train_steps_per_second': 0.381, 'total_flos': 183713587200.0, 'train_loss': 3.800134801864624, 'epoch': 0.01})

In [33]:
peft_model_id = "peft_model_save"

model.save_pretrained(peft_model_id)

In [35]:
%cd 'peft_model_save'
!ls -lha
%cd ..

/content/peft_model_save
total 600K
drwxr-xr-x 2 root root 4.0K Jul  2 20:37 .
drwxr-xr-x 1 root root 4.0K Jul  2 20:37 ..
-rw-r--r-- 1 root root  394 Jul  2 20:41 adapter_config.json
-rw-r--r-- 1 root root 581K Jul  2 20:41 adapter_model.bin
-rw-r--r-- 1 root root  111 Jul  2 20:41 README.md
/content


In [36]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path, return_dict=True, device_map="cpu"
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
batch = tokenizer("Two things are infinite: ", return_tensors="pt")

output_tokens = model.generate(**batch, max_new_tokens=50)

print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 3>:3                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/peft/peft_model.py:912 in generate                       │
│                                                                                                  │
│    909 │   │   else:                                                                             │
│    910 │   │   │   self.base_model.generation_config = self.generation_config                    │
│    911 │   │   try:                                                                              │
│ ❱  912 │   │   │   outputs = self.base_model.generate(**kwargs)                                  │
│    913 │   │   except:                                                                           │
│    914 │   │   │   self.base_model.prepare_inputs_for_generation = self.base_model_prepare_inpu  │
│    915 │   │   │   raise                                                                         │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py:115 in decorate_context       │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1527 in generate        │
│                                                                                                  │
│   1524 │   │   │   │   )                                                                         │
│   1525 │   │   │                                                                                 │
│   1526 │   │   │   # 11. run greedy search                                                       │
│ ❱ 1527 │   │   │   return self.greedy_search(                                                    │
│   1528 │   │   │   │   input_ids,                                                                │
│   1529 │   │   │   │   logits_processor=logits_processor,                                        │
│   1530 │   │   │   │   stopping_criteria=stopping_criteria,                                      │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2349 in greedy_search   │
│                                                                                                  │
│   2346 │   │   │   model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)  │
│   2347 │   │   │                                                                                 │
│   2348 │   │   │   # forward pass to get next token                                              │
│ ❱ 2349 │   │   │   outputs = self(                                                               │
│   2350 │   │   │   │   **model_inputs,                     